In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import requests


def getData(ticker='X:BTCUSD', count=1, timespan='minute', limit=50000):
    api_key = 'q4AAVje7SoHY939BYwnUjpO8Q1o6fSjp'
    to_date = datetime.now() 
    from_date = to_date - timedelta(days=1) 
    to_date = to_date.strftime('%Y-%m-%d') 
    from_date = from_date.strftime('%Y-%m-%d')
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/{count}/{timespan}/{from_date}/{to_date}?adjusted=true&sort=asc&limit={limit}&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data['results'])
    df['timestamp'] = pd.to_datetime(df['t'], unit='ms')
    df.rename(columns={'o': 'Open', 'h': 'High', 'l': 'Low', 'c': 'Close', 'v': 'Volume'}, inplace=True)
    df.drop(columns=['vw', 'n', 't'], inplace=True)
    return df

onedf = getData() # pandas dataframe with candle high, low, open, close
seconddf = getData('X:BTCUSD', 5, 'second') # pandas dataframe with candle high, low, open, close

,Volume,Open,Close,High,Low,timestamp
0,6.926938,96536.93,96402.74,96579.0,96402.74,2025-02-08 00:00:00
1,3.138690,96405.99,96474.38,96508.0,96397.37,2025-02-08 00:01:00
2,1.553925,96480.61,96469.93,96543.0,96446.44,2025-02-08 00:02:00
3,2.737826,96469.37,96419.80,96534.0,96403.21,2025-02-08 00:03:00
4,3.600032,96419.80,96500.00,96534.0,96419.80,2025-02-08 00:04:00
...,...,...,...,...,...,...
1435,6.902669,96588.34,96481.29,96685.0,96481.28,2025-02-08 23:55:00
1436,2.963106,96488.06,96503.30,96611.0,96453.86,2025-02-08 23:56:00
1437,2.197687,96503.29,96486.87,96598.0,96486.85,2025-02-08 23:57:00
1438,7.622836,96486.89,96462.38,96598.0,96459.84,2025-02-08 23:58:00


In [ ]:

onetrend = 0 # predetermined input to start; 1 is up, 0 is down
secondtrend = 0

onehigh = onedf.iloc[0] # last valid high
onelow = onedf.iloc[0] # last valid low
secondhigh = seconddf.iloc[0]
secondlow = seconddf.iloc[0] 

onepossible_high = CANDLE #potential high
onepossible_low = CANDLE #potential low
secondpossible_high = CANDLE 
secondpossible_low = CANDLE

one_last_candle = (candle_high, candle_low) # remember last candle for closure outside
one_curr_candle = (candle_close) #keep track of the current important candle
second_last_candle = ()
second_curr_candle = ()

onesignals = []
secondsignals = []

finalSignals = []

def structure(): # trends with swings
    if uptrend and curr_candle closes below last candle:
        track possible_high and possible_low
        if curr_candle closes above possible high:
            set new valid high and valid low
            signal()
        if curr_candle closes below valid low:
            set new valid high and valid low
            change to downtrend
            signal()
    if uptrend and curr_candle closes above:
        set new possible_high

    if downtrend and curr_candle closes below above candle:
        keep track of possible_low and possible_high
        if curr_candle closes below possible_low:
            set new valid high and valid low
            signal(trend)
        if curr_candle closes above valid_high:
            set new valid high and valid low
            change to uptrend
            signal(trend)
    if downtrend and curr_candle closes below:
        set new possible_low

def signal():
    zone_entry_buy = (valid high - valid_low / 2) + valid_low
    zone_entry_low = (valid_low - valid_high / 2) + valid_high

def finalSignal():
    if uptrend and curr_candle closes below zone_entry_buy:
        buy(valid_low, valid_high)
    if downtrend and curr_candle closes above zone_entry_low:
        sell(valid_high, valid_low)

def buy(stoploss, takeProfit):


def sell(stoploss, takeProfit):


def run():
    if price gets to a signal:
        buy() or sell()